In [190]:
function frag_vec(n, dt,  fragmentation_timescale, mode)
    
    fragmentation_mode = ones(n)
    frag_prob = exp(-abs(dt)/(fragmentation_timescale))
    frag_prob = ones(n)*frag_prob
    rnd_num = rand(n)
    
    truth = rnd_num  .> frag_prob
    
    if mode == "rand"
        fragmentation_mode = rand(n).*truth + .!truth
        
    elseif mode == "Mika"
        raw_mode = rand([8, 4, 2, 1], n) #p=np.array([8/14.5, 4/14.5, 2/14.5, 0.5/14.5])) # FIX this!!
        fragmentation_mode = 1 ./raw_mode.*truth + .!truth
    
    end
    
    return fragmentation_mode
end

frag_vec (generic function with 1 method)

In [187]:
a = frag_vec(10, 0.1, 100, "Mika")

Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

10-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [169]:
function sinkingVelocity(radius, β, ν)
    alpha = radius
    g = 9.81
  
    tau_p = alpha.^2/(3*β*ν)
    
    v_s = (1 - β)*g*tau_p
    
    return v_s
    
end

sinkingVelocity (generic function with 1 method)

In [182]:
sinkingVelocity([1,1,1], 0.8, 1e-6)*.1

3-element Vector{Float64}:
 81749.99999999999
 81749.99999999999
 81749.99999999999

In [203]:
# parameters
r = 5e-5 # 100nm
k = 0
sim_time = 1000 #365*10
N = sim_time + 1
V = 4/3 *π*r^3
mode = 1/2
frag_timescale = 5000
frag_mode = 1/2
k = 0
particles = 10000
initial_depth = 2000
depth_lim = 1500
diameter_lim = 1e-3
β = 0.8
ν = 1e-6

# Da Arrais
time = LinRange(0, sim_time, N)
dt = time[2] - time[1]

lengthxx = zeros(particles)
depthxx = zeros(particles) .+ initial_depth
agexx = zeros(particles)

lengthxx[:] .= r
online = sum(depthxx .> depth_lim)

i = 0
while online != 0
    truth = depthxx .> depth_lim
    length_lim = lengthxx .< diameter_lim
    
    depthxx = depthxx - sinkingVelocity(lengthxx, β, ν)*dt.*truth
    modexx = frag_vec(particles, dt, frag_timescale, "Mika")
    lengthxx = lengthxx.*(1 .+ (1 ./modexx .- 1).*length_lim)
    agexx = agexx + 1 .*truth
    online = sum(truth)
    i += 1
end

In [204]:
agexx

10000-element Vector{Float64}:
 11735.0
  8171.0
  7645.0
  6032.0
 20325.0
  6868.0
 15595.0
 20389.0
 17148.0
 15717.0
  7746.0
 14078.0
 17375.0
     ⋮
  2452.0
 16032.0
 16152.0
 23974.0
  1196.0
 12679.0
 14493.0
 13326.0
  5386.0
  4967.0
  3960.0
  6597.0

In [153]:
LinRange(0, 10, 1000) 

1000-element LinRange{Float64, Int64}:
 0.0,0.01001,0.02002,0.03003,0.04004,…,9.95996,9.96997,9.97998,9.98999,10.0